# Clustering
This segment of the tutorial will teach you how to preform waveform similarity clustering in detex. The function used to preform clustering is createCluster of the construct module. The results are then stored in an instance of the ClusterStream class. Let's start by looking at the createCluster function and some important parameters that go into it, then we will look at the ClusterStream class its methods.

## CreateCluster


In [ ]:
import detex
version = detex.__version__
print ("Detex version is %s\n" % version)
print (detex.construct.createCluster.__doc__)

As you can see there are a lot of input arguments, and a lot to think about when creating a cluster object. Let me elaborate on some of the arguments you should pay special attention to. 

* fet_arg - make sure to look at the detex.getdata.quickFetch docs for this one. Basically, if you want to use a custom DataFetcher be sure to pass it to the createCluster call here or else detex will try to use a local directory with the default name of EventWaveForms.

* filt - parameters to apply a bandpass filter to the waveform similarity clustering and ALL all detex downstream operations. Make sure to think about this carefully before simply using the default, as the default values are not appropriate for all data sets.

* fillZeros - a parameter for handling data with gaps. If data are not avaliable for the entire range (defined by template key and trim parameter) detex will simply fill zeros so that each trace will have the length defined by the trim parameter. The created cluster instance can then be used later on by detex, although you should be careful going forward to no include a bunch of the zero data in your detector, more on that later. 

* trim - a two element list that defines the length of each waveform. The first element is the time before the origin (as reported in the station key) and the second element is the number of seconds after the reported origin time. 

### Dealing with gaps

In order to see how some of these parameters affect the clustering process we will look at an early UUSS dataset that has some issues with gaps. Here are the stations and templates:



In [ ]:
stakey = detex.util.readKey('StationKey.csv', key_type='station')
stakey

In [ ]:
temkey = detex.util.readKey('TemplateKey.csv', key_type='template')
temkey

Because there are so many events it may take some time to get the data. We will skip getting the continuous data because it is not needed for this section of the tutorial. We should probably also start the logger in case we need more info than what is printed to the screen. We will also delete an old logger if there is one.

In [ ]:
import os
if os.path.exists("detex_log.log"):
    os.remove("detex_log.log")
detex.setLogger()
detex.getdata.makeDataDirectories(getContinuous=False)

Now we will cluster these events while varying the input arguments. Let's start by using the defaults.

In [ ]:
%time cl = detex.createCluster() # notice we can call createCluster from the detex level

We see the wall time for the createCluster call was around 2 minutes (on my computer). Let's make a function to see how many of the original 220 events were actually used

In [ ]:
def check_cluster(cl):
    for c in cl:
        sta = c.station
        num_events = len(c.key)
        print '%s had %d events used in the analysis' % (sta, num_events)
    print '\n'
def get_unused_events(cl, temkey):
    for c in cl:
        sta = c.station
        unused = list(set(temkey.NAME) - set(c.key))
        print 'Unused events on %s are:\n %s\n' % (sta, unused)

def get_info(cl, temkey_in='TemplateKey.csv'):
    temkey = detex.util.readKey(temkey_in, 'template')
    print 'There are %d events in the template key' % len(temkey) 
    check_cluster(cl)
    get_unused_events(cl, temkey)

get_info(cl)

Now let's try using fillZeros as True rather than the default of False. This will force each event waveform to be exactly the length defined by the trim parameter by filling with zeros where necessary.  

In [ ]:
%time cl2 = detex.createCluster(fillZeros=True)

In [ ]:
get_info(cl2)

So setting fill_zeros to True caused detex to use all the events on MSU and all but four on IMU. The four IMU events that went unused were probably due to missing waveforms. We can verify this by looking in the log for indications the that the data were not available to download.

In [ ]:
log = detex.util.readLog()
log

### Time Trials
If you are trying to perform waveform clustering on a large data set it may be worth your time to understand how varying certain parameters can affect runtimes. Let's isolate a few variables and compare run times from the default values. If you are running this on your computer at home it may take some time, skip ahead if you aren't interested. 

In [ ]:
# Setup code for time trials
import time
def timeit(func): # decorator for timing function calls
    def wraper(*args, **kwargs):
        t = time.time()
        out = func(*args, **kwargs)
        return (time.time() - t, out)
    return wraper

@timeit
def time_cluster(*args, **kwargs):
    detex.createCluster(*args, **kwargs)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
detex.verbose = False # silence detex

cols = ['waveform_duration', 'run_time']
df = pd.DataFrame(columns=cols)

trims = [(10, 120), (5, 60), (2, 30), (1, 15)]

for trim in trims:
    rt = time_cluster(trim=trim)[0]
    ser = pd.Series([sum(trim), rt], index=cols)
    df.loc[len(df)] = ser
    
plt.plot(df.waveform_duration, df.run_time)
plt.title("Waveform Length vs Run Times")
plt.ylabel("run times (seconds)")
plt.xlabel("waveform lengths (seconds)")

    
    

In [ ]:
cols = ['num_events', 'run_time']
df = pd.DataFrame(columns=cols)

temkey = detex.util.readKey("TemplateKey.csv", "template")

temkey_lengths = [10, 20, 50, 100, 150, 200]

for tkl in temkey_lengths:
    temkey2 = temkey.copy()
    
    rt = time_cluster(templateKey=temkey2[:tkl+1])[0]
    ser = pd.Series([tkl, rt], index=cols)
    df.loc[len(df)] = ser
    
plt.plot(df.num_events, df.run_time)
plt.title("Number of Events vs Runtimes")
plt.xlabel("Number of Events")
plt.ylabel("Runtimes (seconds)")




Although a bit more complicated than this, we could qualitatively estimate that changing the waveform length scales the runtime by approximately N (linearly with time) whereas the number of events scales the runtime by approximately N<sup>2</sup> (quadratic with time). Let's see how decimating the data changes the runtimes. 


In [ ]:
# Test various decimation factors
rt_base = time_cluster()[0]
rt_decimate = time_cluster(decimate=2)[0]
print("Base run time: %.02f, Decimated run time: %.02f" % (rt_base, rt_decimate))


Interestingly, this didn't seem to make much of a difference. The original data were sampled at 100 Hz so using a decimation factor of 2 would have reduced the sampling rate to 50 Hz. Since we left the default bandpass filter (1.0 to 10.0 Hz) it might make sense to use a decimation factor of 4 in order to bring the sampling rate down to 25 Hz. 

## ClusterStream and Cluster Classes

The ClusterStream and Cluster classes are used to control and visualize waveform similarity clustering. These classes are required to define the subspaces used in the detection process.

The ClusterStream is a container for one or more Cluster instances. There is a cluster instance for each station, although most attributes are accessible from the ClusterStream level. Let's take create a ClusterStream instance and take a closer look.  

In [ ]:
import detex # reimport so we can start here
detex.verbose = False
cl = detex.createCluster()

The bulk of the information for the ClusterStream is stored in the trdf attribute, which, of course, is a pandas DataFrame.

In [ ]:
cl.trdf

In this DataFrame there is a row for each station. The columns are:

| Column | Description |
|:-----:| :---------: |
| CCs | A matrix of max correlation coef for each station pair |
| Lags | A matrix of lag samples corresponding to the highest correlation coef |
| Subsamp | The decimal fraction determined by subsample extrapolation |
| Events | The name of the events used |
| Stats | Selected stats of the events |

The CCs and Lags are DataFrames that have indices and rows that correspond to an element in the Events list. This is probably best illustrated by an example. Let's say we want to find the max correlation ceof. between two events and the corresponding number of samples that would be required to shift the first event to line up with the second. First, we need to find where the events we want to find occur in the events list, then we can index them in the lags and ccs.


In [ ]:
# Here are two events in the list
ev1 = '2010-07-10T08-57-51.25'
ev2 = '2014-11-29T14-18-04.87'
events = list(cl.trdf.loc[0, 'Events']) # cast from np array to list
# Find the index where each event occurs in the list
ev1_ind = events.index(ev1)
ev2_ind = events.index(ev2)
print ("%s index is %d, %s index is %d" % (ev1, ev1_ind, ev2, ev2_ind))



In [ ]:
cc = cl.trdf.loc[0, 'CCs']
lags = cl.trdf.loc[0, 'Lags']
coef = cc.loc[ev1_ind, ev2_ind]
lag = lags.loc[ev1_ind, ev2_ind]
print (coef, lag)
# events

### Visualization Methods
The ClusterStream has several methods for visualizing. We can create a simple similarity matrix.

In [ ]:
cl.simMatrix()

By default the events (x and y axis) are ordered based on origin time. We can also plot them based on the groups the events best fit in. 

In [ ]:
cl.simMatrix(groupClusts=True)

We can visualize and change the clustering structure for each station with the dendro and updateReqCC methods, just as in the intro tutorial. 

In [ ]:
cl.dendro()

In [ ]:
cl[0].updateReqCC(.6)
cl[0].dendro()

We can plot the spatial relations of the events with the plotEvents method. This is used to get a quick and dirty idea of event locations and depths; it still needs a lot of work before it will produce presentable plots. The following is not the best example of a meaningful plot because there are so many colors and different groups but plotEvents can be useful, especially on smaller datasets. 

In [ ]:
cl[0].plotEvents()

# Next section
The [next section](../SubspaceDetection/subspace_detection1.md) covers subspace detection.